In [1]:
import pickle
import numpy as np
from collections import defaultdict

# --- USER SETTING ---
PERCENTILE = 91  # flag all windows above this percentile (e.g. 95th percentile)

# --- Load Results ---
with open('SMD500_2.pkl', 'rb') as f:
    all_results = pickle.load(f)

# --- Compute Anomaly Scores ---
anomaly_scores = defaultdict(lambda: {'score_sum': 0.0, 'count': 0})

for iteration_result in all_results:
    buckets = iteration_result['buckets']
    bucket_results = iteration_result['bucket_results']

    for bucket_result in bucket_results:
        bucket_idx = bucket_result['bucket_idx']
        final_results = bucket_result['final_results']
        indices_in_bucket = buckets[bucket_idx]

        mean = np.mean(final_results)
        std = np.std(final_results) if np.std(final_results) != 0 else 1e-8

        for i, idx in enumerate(indices_in_bucket):
            sim = final_results[i]
            deviation = abs(sim - mean) / std
            anomaly_scores[idx]['score_sum'] += deviation
            anomaly_scores[idx]['count'] += 1

# --- Final Score Calculation ---
final_scores = {
    idx: score_data['score_sum'] / score_data['count']
    for idx, score_data in anomaly_scores.items()
}

# --- Compute Threshold by Percentile ---
all_score_values = list(final_scores.values())
threshold = np.percentile(all_score_values, PERCENTILE)

# --- Flag Anomalous Windows ---
anomalous_windows = {
    idx: score for idx, score in final_scores.items() if score >= threshold
}

print(f"\n{len(anomalous_windows)} anomalous windows detected above the {PERCENTILE}th percentile (threshold ≈ {threshold:.4f}):\n")
for idx, score in sorted(anomalous_windows.items()):
    print(f"Window {idx} - Score: {score:.4f}")

# --- Save to file ---
with open("SMD500_2", "w") as f:
    for idx, score in sorted(anomalous_windows.items()):
        f.write(f"Window {idx} - Score: {score:.4f}\n")

print("\nSaved to 'flagged_anomalous_windows.txt'")



52 anomalous windows detected above the 91th percentile (threshold ≈ 0.8481):

Window 123 - Score: 0.8512
Window 154 - Score: 0.8511
Window 241 - Score: 0.8598
Window 247 - Score: 0.8940
Window 248 - Score: 0.9378
Window 249 - Score: 0.8973
Window 250 - Score: 0.8676
Window 274 - Score: 0.9227
Window 275 - Score: 0.9794
Window 276 - Score: 0.9696
Window 277 - Score: 0.8758
Window 303 - Score: 0.9546
Window 304 - Score: 0.9904
Window 305 - Score: 0.8936
Window 316 - Score: 0.8530
Window 318 - Score: 0.8973
Window 319 - Score: 0.9681
Window 321 - Score: 0.8521
Window 322 - Score: 0.8858
Window 324 - Score: 0.8819
Window 325 - Score: 0.9245
Window 326 - Score: 0.9228
Window 327 - Score: 0.8527
Window 338 - Score: 0.8645
Window 345 - Score: 0.8539
Window 346 - Score: 1.0628
Window 347 - Score: 1.3951
Window 348 - Score: 1.5717
Window 349 - Score: 1.4122
Window 350 - Score: 0.9006
Window 361 - Score: 0.9146
Window 362 - Score: 0.9008
Window 364 - Score: 0.8890
Window 365 - Score: 0.9162
Wi